In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('/content/Creditcard_data.csv')

data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [2]:
data.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [3]:
x=data.drop('Class',axis=1)
y=data['Class']

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [5]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)
X_rus, y_rus = rus.fit_resample(X_train, y_train)

In [6]:
from imblearn.over_sampling import SMOTE, RandomOverSampler

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X_train, y_train)


In [7]:
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=42)
X_cc, y_cc = cc.fit_resample(X_train, y_train)

In [8]:
from imblearn.combine import SMOTETomek

st = SMOTETomek(random_state=42)
X_st, y_st = st.fit_resample(X_train, y_train)

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

models = {
    'M1': LogisticRegression(max_iter=1000),
    'M2': RandomForestClassifier(n_estimators=100),
    'M3': SVC(probability=True),
    'M4': KNeighborsClassifier(),
    'M5': GradientBoostingClassifier()
}



In [10]:
from sklearn.metrics import accuracy_score

# Create a dictionary to store accuracy scores
results = {}

# Train each model on each sample and calculate the accuracy
for sample_name, data in [('Random Under-Sampling', (X_rus, y_rus)),
                          ('Random Over-Sampling', (X_ros, y_ros)),
                          ('SMOTE', (X_smote, y_smote)),
                          ('Cluster Centroids', (X_cc, y_cc)),
                          ('SMOTETomek', (X_st, y_st))]:
    sample_results = []
    for model_name, model in models.items():
        X_sample, y_sample = data
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)
        model.fit(X_train_sample, y_train_sample)
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        sample_results.append(accuracy)
    results[sample_name] = sample_results

results_df = pd.DataFrame(results, index=models.keys())



In [11]:
results_df

,Random Under-Sampling,Random Over-Sampling,SMOTE,Cluster Centroids,SMOTETomek
M1,0.827586,0.922414,0.918103,0.482759,0.918103
M2,0.767241,0.987069,0.987069,0.655172,0.987069
M3,0.987069,0.706897,0.426724,0.668103,0.521552
M4,0.625000,0.978448,0.689655,0.616379,0.676724
M5,0.689655,0.969828,0.965517,0.508621,0.969828
